<a href="https://colab.research.google.com/github/komo135/tensorflow-rl/blob/master/ipynb/qr_dqn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Google ドライブをマウントするには、このセルを実行してください。
from google.colab import drive
drive.mount('/content/drive')

%cd drive/My Drive

Mounted at /content/drive
/content/drive/My Drive


In [1]:
%tensorflow_version 2.x
import tensorflow as tf

TensorFlow 2.x selected.


In [6]:
%cd /content/drive/My Drive

try:
  import imp
  imp.reload(qr_dqn)
except:
  import qr_dqn

agent = qr_dqn.Agent(spread=10, pip_cost=1000, leverage=500, min_lots=0.01, assets=10000, available_assets_rate=0.4,
                 restore=True, step_size=96, n=4, lr=1e-3)

/content/drive/My Drive


In [0]:
%cd /content
agent.run()

/content
action probability: buy=0.4166666666666667, sell=0.5833333333333334, hold=0.0
epoch: 59000, total assets:[1966.35647318], growth_rate:[0.19663565]

action probability: buy=0.4583333333333333, sell=0.5416666666666666, hold=0.0
epoch: 59005, total assets:[1917.14804784], growth_rate:[0.1917148]

action probability: buy=0.5416666666666666, sell=0.4583333333333333, hold=0.0
epoch: 59010, total assets:[1372.45195094], growth_rate:[0.1372452]

action probability: buy=0.3958333333333333, sell=0.6041666666666666, hold=0.0
epoch: 59015, total assets:[1450.01794948], growth_rate:[0.14500179]

action probability: buy=0.4375, sell=0.5625, hold=0.0
epoch: 59020, total assets:[3310.28609032], growth_rate:[0.33102861]

action probability: buy=0.46875, sell=0.53125, hold=0.0
epoch: 59025, total assets:[3007.10217984], growth_rate:[0.30071022]

action probability: buy=0.5208333333333334, sell=0.4791666666666667, hold=0.0
epoch: 59030, total assets:[11731.16777475], growth_rate:[1.17311678]

ac

In [0]:
import numpy as np
tree_idx, replay = agent.memory.sample(128)

states = np.array([a[0][0] for a in replay], np.float32)
new_states = np.array([a[0][3] for a in replay], np.float32)
actions = np.array([a[0][1] for a in replay]).reshape((-1, 1))
rewards = np.array([a[0][2] for a in replay], np.float32).reshape((-1, 1))

q = agent.model.predict_on_batch(states)
target_q = agent.target_model.predict_on_batch(new_states).numpy()
arg_q = np.sum(agent.model.predict_on_batch(new_states).numpy(), -1).reshape((-1,2))
arg_q = [np.argmax(i) if 0.05 < np.random.rand() else np.random.randint(2) for i in arg_q]

q_backup = q.numpy()

for i in range(q.shape[0]):
  q_backup[i, actions[i]] = rewards[i] + agent.gamma * target_q[i, arg_q[i]]

loss = agent.huber_loss(q_backup, q)
loss = tf.abs(agent.tau - tf.stop_gradient(tf.cast((q_backup - q) < 0, tf.float32))) * loss
loss = tf.reduce_sum(loss, 2)

# print(q)

In [12]:
np.sum(agent.model.predict_on_batch(new_states), -1)

array([[ -7431.06   ,  -7717.0967 ],
       [ -7971.8975 ,  -7195.4453 ],
       [ -5939.387  ,  -5821.9443 ],
       [ -8644.045  ,  -8245.766  ],
       [-23886.777  , -11906.374  ],
       [ 30714.691  , -17660.312  ],
       [  -512.0879 , -10413.939  ],
       [ -2405.787  ,  -7520.604  ],
       [-30555.559  ,   5074.3945 ],
       [ -5154.1807 ,  -5141.465  ],
       [  -180.75293,    943.1699 ],
       [  -409.04297,  -1692.4199 ],
       [-20057.5    , -25640.059  ],
       [  3383.8008 ,   3356.9507 ],
       [ -3107.9717 , -24264.725  ],
       [ -4596.301  ,   2077.2021 ],
       [-42661.418  , -51958.4    ],
       [-31935.365  , -26223.484  ],
       [-25629.531  , -13863.772  ],
       [-17389.13   ,  -2244.3438 ],
       [-23054.234  , -20371.473  ],
       [  2394.255  ,  14088.957  ],
       [  2216.8896 ,   7157.5703 ],
       [-22253.938  , -16932.273  ],
       [-21468.398  , -12227.379  ],
       [-25843.186  , -26459.37   ],
       [ -1948.2769 ,  -4112.077  ],
 

In [9]:
q

<tf.Tensor: shape=(128, 2, 200), dtype=float32, numpy=
array([[[-2299.9995 ,  -400.24338,  -340.95947, ...,   276.5748 ,
           324.45236,   433.5876 ],
        [-2125.1494 ,  -401.49814,  -335.21808, ...,   253.02989,
           300.8101 ,   403.3588 ]],

       [[-2158.723  ,  -373.4729 ,  -314.1767 , ...,   251.40752,
           293.11   ,   393.3672 ],
        [-2000.9341 ,  -359.82037,  -296.06207, ...,   233.40987,
           285.54636,   375.91013]],

       [[-1840.1577 ,  -366.04285,  -315.1082 , ...,   169.21529,
           211.54889,   307.4104 ],
        [-1636.7559 ,  -355.25775,  -307.7799 , ...,   251.71346,
           286.45505,   384.18878]],

       ...,

       [[-2297.665  ,  -430.36688,  -366.6861 , ...,   238.00822,
           290.89343,   406.46527],
        [-2090.0156 ,  -417.3941 ,  -349.87845, ...,   306.69202,
           350.4634 ,   455.90555]],

       [[-2715.0752 ,  -497.70868,  -415.61346, ...,   370.63824,
           441.22388,   559.1806 ],
      